Preparing "worst" and "best" subsets per category in dataset bases on hash distance metrics

In [1]:
import numpy as np
from cifar10_trainer import get_cat
#see create_training_paths for creation of the precalculated dcts; 
#due to copyright these may present an issue when uploaded/clash with the open license -> you have to repeat the script yourself
cifar10_hashes = np.load('cifar10_hashes.npz')
hashes, paths = cifar10_hashes['hashes'], cifar10_hashes['paths']
#calculate start/stop idx per category (should be in order val->train and per set in order of categories)
idx = {}
for i,p in enumerate(paths[10000:]):
    idx.setdefault(get_cat(p),[]).append(i+10000)
idx = {k:[min(v),max(v)+1] for k,v in idx.items()}

Helper functions to calculate hamming distance for numpy np.uint8 arrays

In [82]:
bit_counts = np.array([int(bin(x).count("1")) for x in range(256)]).astype(np.uint8)
def hamming_dist(a,b,axis=None):
    return np.sum(bit_counts[np.bitwise_xor(a,b)],axis=axis)
def hex2hash(s):
    return np.frombuffer(bytes.fromhex(s), dtype=np.uint8)
def hash2hex(h):
    return h.tobytes().hex()
def calc_inter_dist(hashes, eye_val=1024):
    all_d = np.zeros((hashes.shape[0],hashes.shape[0]),dtype=np.int32)
    for i in range(hashes.shape[0]):
        all_d[i,i+1:] = hamming_dist(hashes[i],hashes[(i+1):], axis=1)
    return all_d+all_d.T+np.eye(hashes.shape[0],dtype=np.int32)*eye_val

This method calculates the "best" and "worst" subsets. See paper for description. 

The parameters allow additional subset heuristics to be tested

In [597]:
def min_max_list(dists_sqr, comb_vers=0):
    max_idx = dists_sqr.shape[0]
    if comb_vers < 2:
        dists_fixed = dists_sqr*(1-np.eye(max_idx,max_idx,dtype=np.int32))
    else:
        max_val = np.max(dists_sqr)+1
        dists_fixed = dists_sqr+np.eye(max_idx,max_idx,dtype=np.int32)*max_val
    all_idx, fixed_idx = list(range(max_idx)), []
    while len(fixed_idx) < max_idx:
        if len(fixed_idx) == 0:
            open_list = all_idx
            check_list = dists_fixed
        else:
            open_list = list(set(all_idx).difference(set(fixed_idx)))
            check_list = dists_fixed[fixed_idx][:,open_list]
        if comb_vers == 0:
            idx0 = np.argmin(np.max(check_list, axis = 0))
        elif comb_vers == 1:
            idx0 = np.argmax(np.max(check_list, axis = 0))
        elif comb_vers == 2:
            idx0 = np.argmin(np.min(check_list, axis = 0))
        else:
            idx0 = np.argmax(np.min(check_list, axis = 0))
        idx1 = open_list[idx0]
        fixed_idx.append(idx1)
    return fixed_idx

#apply "best"/"worst" heuristic individually per class
def get_minimal_sets(hashes, idx, comb_vers = 0):
    ret_sets = {}
    for k,idx_m in idx.items():
        dists32 = calc_inter_dist(hashes[idx_m[0]:idx_m[1]], 0)
        dists_sqr = dists32*dists32
        ret_sets[k] = [i+idx_m[0] for i in min_max_list(dists_sqr, comb_vers=comb_vers)]
    return ret_sets

# for comparision/reference calculate 10 random subsets
def get_rand_seq(idx):
    retseq = {}
    for k, v in idx.items():
        l0 = list(range(v[0],v[1]))
        np.random.shuffle(l0)
        retseq[k] = np.int32(l0).tolist()
    return retseq


worstset = get_minimal_sets(hashes, idx, comb_vers = 0)
bestset = get_minimal_sets(hashes, idx, comb_vers = 3)
randsets = {'r%i'%i:get_rand_seq(idx) for i in range(10)}

The subset paths are now stored in a json file (indices need conversions as json does not store np.int64 values)

In [490]:
import json
hashsets = {'worst': worstset, 'best': bestset}
hash_sets_int ={k0:{k1:[int(i) for i in v1] for k1,v1 in v0.items()} for k0,v0 in hash_sets.items()}
hash_sets_int['paths'] = paths.tolist()
hash_sets_int.update(randsets)
json.dump(hash_sets_int, open('cifar10_hashsets.json','wt'))
!gzip cifar10_hashsets.json